# ODIN in WFM mode

This is a simulation of the ODIN chopper cascade in WFM mode.
We also show how one can convert the neutron arrival times at the detector to wavelength.

In [ ]:
import scipp as sc
import plopp as pp
import tof

## Create a source pulse

We first create a source with 4 pulses containing 500,000 neutrons each,
and whose distribution follows the ESS time and wavelength profiles (both thermal and cold neutrons are included).

In [ ]:
source = tof.Source(facility="ess", neutrons=500_000, pulses=4)
source.plot()

## Component set-up

The ODIN chopper cascade consists of:

- 2 WFM choppers
- 5 frame-overlap choppers
- 2 band-control choppers
- 1 T0 chopper

It also has a single detector panel 60.5 meters from the source.

These component parameters are pre-configured in `tof.facilities.ess`.
Here we select the configuration in pulse-skipping mode:

In [ ]:
odin_params = tof.facilities.ess.odin(pulse_skipping=True)
odin_params

## Run the simulation

We propagate our pulse of neutrons through the chopper cascade and inspect the results.

In [ ]:
model = tof.Model(source=source, **odin_params)
results = model.run()
results.plot(blocked_rays=5000)

We can see that the chopper cascade is implementing WFM and pulse-skipping at the same time!

## Wavelength as a function of time-of-arrival

### Plotting wavelength vs time-of-arrival

Since we know the true wavelength of our neutrons,
as well as the time at which the neutrons arrive at the detector
(coordinate named `toa` in the detector reading),
we can plot an image of the wavelengths as a function of time-of-arrival:

In [ ]:
# Squeeze the pulse dimension since we only have one pulse
events = results['detector'].data.flatten(to='event')
# Remove the events that don't make it to the detector
events = events[~events.masks['blocked_by_others']]
# Histogram and plot
events.hist(wavelength=500, toa=500).plot(norm='log', grid=True)

### Defining a conversion from `toa` to `wavelength`

The image above shows that there is a pretty tight correlation between time-of-arrival and wavelength.

We compute the mean wavelength inside a given `toa` bin to define a relation between `toa` and `wavelength`.

In [ ]:
binned = events.bin(toa=500)

# Weighted mean of wavelength inside each bin
mu = (
    binned.bins.data * binned.bins.coords['wavelength']
).bins.sum() / binned.bins.sum()

# Variance of wavelengths inside each bin
var = (
    binned.bins.data * (binned.bins.coords['wavelength'] - mu) ** 2
) / binned.bins.sum()

We can now overlay our mean wavelength function on the image above:

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2, 1)

f = events.hist(wavelength=500, tof=500).plot(norm='log', cbar=False, ax=ax[0])
mu.name = 'Wavelength'
mu.plot(ax=ax[0], color='C1', grid=True)
stddev = sc.sqrt(var.hist())
stddev.name = 'Standard deviation'
stddev.plot(ax=ax[1], grid=True)
fig.set_size_inches(6, 8)
fig.tight_layout()

## Computing wavelengths

We set up an interpolator that will compute wavelengths given an array of `toas`.

In [ ]:
from scipp.scipy.interpolate import interp1d

# Set up interpolator
y = mu.copy()
y.coords['toa'] = sc.midpoints(y.coords['toa'])
f = interp1d(y, 'toa', bounds_error=False)

# Compute wavelengths
wavs = f(events.coords['toa'].rename_dims(event='toa'))
wavelengths = sc.DataArray(
    data=sc.ones(sizes=wavs.sizes, unit='counts'), coords={'wavelength': wavs.data}
).rename_dims(toa='event')
wavelengths

We can now compare our computed wavelengths to the true wavelengths of the neutrons:

In [ ]:
pp.plot(
    {
        'wfm': wavelengths.hist(wavelength=300),
        'original': events.hist(wavelength=300),
    }
)